<a href="https://colab.research.google.com/github/SurajKumar-27/ML-projects/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Stop words in English

In [4]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [8]:
news_dataset=pd.read_csv('/content/train.csv')

In [9]:
news_dataset.shape

(20800, 5)

In [10]:
news_dataset.head

<bound method NDFrame.head of           id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2        

In [11]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

Replace null values with empty string

In [12]:
news_dataset=news_dataset.fillna('')

In [13]:
#merging author name and new title
news_dataset['content']=news_dataset['author']+news_dataset['title']
print(news_dataset['content'])

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [15]:
X=news_dataset.drop(columns='label',axis=1)
Y=news_dataset['label']
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [16]:
X.shape

(20800, 5)

Stemming is the process of removing prefix and suffix to give root word

In [17]:
port_stem=PorterStemmer()

In [24]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)#replace non alphabets with space
  stemmed_content = stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [25]:
news_dataset['content']=news_dataset['content'].apply(stemming)

In [26]:
print(news_dataset['content'])

0        darrel lucush dem aid even see comey letter ja...
1        daniel j flynnflynn hillari clinton big woman ...
2                consortiumnew comwhi truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp trump poster child white supr...
20796    benjamin hoffmann f l playoff schedul matchup ...
20797    michael j de la merc rachel abramsmaci said re...
20798    alex ansarynato russia hold parallel exerci ba...
20799                        david swansonwhat keep f aliv
Name: content, Length: 20800, dtype: object


In [27]:
X=news_dataset['content'].values
Y=news_dataset['label'].values
print(X)
print(Y)

['darrel lucush dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynnflynn hillari clinton big woman campu breitbart'
 'consortiumnew comwhi truth might get fire' ...
 'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time'
 'alex ansarynato russia hold parallel exerci balkan'
 'david swansonwhat keep f aliv']
[1 0 1 ... 0 1 1]


In [29]:
#textual data to numerical data
vectorizer=TfidfVectorizer()#Tf-term frequency and idf-inverse document frequency
vectorizer.fit(X)
X=vectorizer.transform(X)
print(X)


  (0, 26171)	0.28088379401596425
  (0, 22568)	0.2552336018069161
  (0, 14920)	0.43006226759639316
  (0, 14460)	0.29177259684200296
  (0, 12695)	0.24619727512767195
  (0, 7961)	0.23133661742488731
  (0, 6233)	0.2839932825877813
  (0, 5930)	0.35488202138141456
  (0, 4975)	0.2472595823572816
  (0, 4187)	0.3625320323150658
  (0, 572)	0.2694167078545385
  (1, 27744)	0.37204408227449065
  (1, 11235)	0.2434639969074374
  (1, 8704)	0.5300315426405939
  (1, 5877)	0.3206218375341294
  (1, 4739)	0.23523739119536424
  (1, 3838)	0.44601381123255807
  (1, 3260)	0.18800278047529806
  (1, 2602)	0.35911932371235855
  (2, 26066)	0.3665032495181434
  (2, 16256)	0.43295215406038445
  (2, 9384)	0.30743020569262086
  (2, 8499)	0.3411947414020896
  (2, 5204)	0.40440534260277944
  (2, 5089)	0.5511414848555652
  :	:
  (20797, 25609)	0.08234347981192172
  (20797, 25155)	0.3125002447493035
  (20797, 21934)	0.2494515865666979
  (20797, 20637)	0.2734270447609076
  (20797, 20353)	0.25042469546773877
  (20797, 17396

Splitting data to train set and test data

In [30]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [31]:
#training the model
model=LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

In [32]:
#Evaluating the model using accuracy score
#accuracy score on training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [33]:
print("Accuracy score of training data: ",training_data_accuracy)

Accuracy score of training data:  0.9719350961538461


In [35]:
#accuracy score on testing data
X_test_prediction=model.predict(X_test)
testing_data_accuracy=accuracy_score(X_test_prediction,Y_test)
print("Accuracy score of testing data: ",testing_data_accuracy)

Accuracy score of testing data:  0.9550480769230769


In [39]:
X_new=X_test[32]
prediction=model.predict(X_new)
print(prediction)
if(prediction==0):
  print("News is real")
else:
  print("News is fake")

[1]
News is fake


In [40]:
print(Y_test[32])

1
